In [273]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
import time # to measure how long the models take
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from imblearn.over_sampling import SMOTE
import math
import statsmodels.api as sm
import os

In [274]:
df_players = pd.read_csv(r"C:\Users\gabri\Documents\player_advanced_stats_data.csv")
df_goalies = pd.read_csv(r"C:\Users\gabri\Documents\goalie_advanced_stats_data.csv")

In [275]:
df_players.head()

,Unnamed: 0,Player,Team,Position,GP,TOI,CF,CA,CF%,FF,...,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %
0,1,Rob Blake,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Rod Brind'Amour,CAR,C,59,1324.550000,1309.0,1161.0,53.0,961.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Patrice Brisebois,MTL,D,43,723.800000,749.0,626.0,54.47,556.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Chris Chelios,DET,D,69,1170.233333,886.0,901.0,49.58,709.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Kris Draper,DET,C,65,1016.116667,694.0,853.0,44.86,551.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [276]:
df_players = df_players.dropna(subset=['Unnamed: 0'])

In [277]:
df_players = df_players[df_players['On Ice Situation'] == 'All Strengths']

In [278]:
df_players = df_players.merge(df_players, on=['Player', 'Years'])

In [279]:
df_players.drop_duplicates(inplace=True)

In [280]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61124 entries, 0 to 61123
Columns: 170 entries, Unnamed: 0_x to Faceoffs %_y
dtypes: float64(116), int64(4), object(50)
memory usage: 79.3+ MB


In [281]:
df_players.head()

,Unnamed: 0_x,Player,Team_x,Position_x,GP_x,TOI_x,CF_x,CA_x,CF%_x,FF_x,...,Misconduct_y,Penalties Drawn_y,Giveaways_y,Takeaways_y,Hits_y,Hits Taken_y,Shots Blocked_y,Faceoffs Won_y,Faceoffs Lost_y,Faceoffs %_y
0,1,Rob Blake,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Rob Blake,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,...,3.0,7.0,58.0,21.0,93.0,25.0,143.0,4.0,4.0,50.00
2,1,Rob Blake,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Rob Blake,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,...,3.0,7.0,58.0,21.0,93.0,25.0,143.0,4.0,4.0,50.00
4,2,Rod Brind'Amour,CAR,C,59,1324.550000,1309.0,1161.0,53.0,961.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [282]:
df_players.iloc[:,:20].head()

,Unnamed: 0_x,Player,Team_x,Position_x,GP_x,TOI_x,CF_x,CA_x,CF%_x,FF_x,FA_x,FF%_x,SF_x,SA_x,SF%_x,GF_x,GA_x,GF%_x,xGF_x,xGA_x
0,1,Rob Blake,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,1266.0,46.54,786.0,891.0,46.87,74.0,105.0,41.34,81.42,93.01
1,1,Rob Blake,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,1266.0,46.54,786.0,891.0,46.87,74.0,105.0,41.34,81.42,93.01
2,1,Rob Blake,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Rob Blake,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Rod Brind'Amour,CAR,C,59,1324.550000,1309.0,1161.0,53.0,961.0,875.0,52.34,698.0,621.0,52.92,69.0,76.0,47.59,66.54,68.57


In [283]:
df_players.iloc[:,20:40].head()

,xGF%_x,SCF_x,SCA_x,SCF%_x,HDCF_x,HDCA_x,HDCF%_x,HDGF_x,HDGA_x,HDGF%_x,MDCF_x,MDCA_x,MDCF%_x,MDGF_x,MDGA_x,MDGF%_x,LDCF_x,LDCA_x,LDCF%_x,LDGF_x
0,46.68,710.0,827.0,46.19,278.0,319.0,46.57,46.0,57.0,44.66,432.0,508.0,45.96,17.0,30.0,36.17,615.0,724.0,45.93,10.0
1,46.68,710.0,827.0,46.19,278.0,319.0,46.57,46.0,57.0,44.66,432.0,508.0,45.96,17.0,30.0,36.17,615.0,724.0,45.93,10.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49.25,582.0,601.0,49.2,253.0,265.0,48.84,42.0,42.0,50.00,329.0,336.0,49.47,13.0,18.0,41.94,606.0,472.0,56.22,13.0


In [284]:
df_players.iloc[:,40:60].head()

,LDGA_x,LDGF%_x,On-Ice SH%_x,On-Ice SV%_x,PDO_x,Off. Zone Starts_x,Neu. Zone Starts_x,Def. Zone Starts_x,On The Fly Starts_x,Off. Zone Start %_x,Off. Zone Faceoffs_x,Neu. Zone Faceoffs_x,Def. Zone Faceoffs_x,Off. Zone Faceoff %_x,Years,On Ice Situation_x,Stats_x,Goals_x,Total Assists_x,First Assists_x
0,15.0,40.00,9.41,88.22,0.976,262.0,416.0,319.0,930.0,45.09,484.0,607.0,551.0,46.76,2007-2008,All Strengths,On-Ice,NaN,NaN,NaN
1,15.0,40.00,9.41,88.22,0.976,262.0,416.0,319.0,930.0,45.09,484.0,607.0,551.0,46.76,2007-2008,All Strengths,On-Ice,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-2008,All Strengths,Individual,9.0,22.0,12.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2007-2008,All Strengths,Individual,9.0,22.0,12.0
4,15.0,46.43,9.89,87.76,0.976,241.0,345.0,269.0,709.0,47.25,438.0,533.0,519.0,45.77,2007-2008,All Strengths,On-Ice,NaN,NaN,NaN


In [285]:
df_players.iloc[:,60:80].head()

,Second Assists_x,Total Points_x,IPP_x,Shots_x,SH%_x,ixG_x,iCF_x,iFF_x,iSCF_x,iHDCF_x,Rush Attempts_x,Rebounds Created_x,PIM_x,Total Penalties_x,Minor_x,Major_x,Misconduct_x,Penalties Drawn_x,Giveaways_x,Takeaways_x
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,31.0,41.89,144.0,6.25,8.84,276.0,199.0,69.0,18.0,6.0,17.0,98.0,37.0,34.0,0.0,3.0,7.0,58.0,21.0
3,10.0,31.0,41.89,144.0,6.25,8.84,276.0,199.0,69.0,18.0,6.0,17.0,98.0,37.0,34.0,0.0,3.0,7.0,58.0,21.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [286]:
df_players.iloc[:,80:100].head()

,Hits_x,Hits Taken_x,Shots Blocked_x,Faceoffs Won_x,Faceoffs Lost_x,Faceoffs %_x,Unnamed: 0_y,Team_y,Position_y,GP_y,TOI_y,CF_y,CA_y,CF%_y,FF_y,FA_y,FF%_y,SF_y,SA_y,SF%_y
0,NaN,NaN,NaN,NaN,NaN,NaN,1,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,1266.0,46.54,786.0,891.0,46.87
1,NaN,NaN,NaN,NaN,NaN,NaN,1,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,93.0,25.0,143.0,4.0,4.0,50.00,1,L.A,D,71,1614.066667,1416.0,1646.0,46.24,1102.0,1266.0,46.54,786.0,891.0,46.87
3,93.0,25.0,143.0,4.0,4.0,50.00,1,L.A,D,71,1614.066667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,2,CAR,C,59,1324.550000,1309.0,1161.0,53.0,961.0,875.0,52.34,698.0,621.0,52.92


In [287]:
df_players.iloc[:,100:120].head()

,GF_y,GA_y,GF%_y,xGF_y,xGA_y,xGF%_y,SCF_y,SCA_y,SCF%_y,HDCF_y,HDCA_y,HDCF%_y,HDGF_y,HDGA_y,HDGF%_y,MDCF_y,MDCA_y,MDCF%_y,MDGF_y,MDGA_y
0,74.0,105.0,41.34,81.42,93.01,46.68,710.0,827.0,46.19,278.0,319.0,46.57,46.0,57.0,44.66,432.0,508.0,45.96,17.0,30.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,74.0,105.0,41.34,81.42,93.01,46.68,710.0,827.0,46.19,278.0,319.0,46.57,46.0,57.0,44.66,432.0,508.0,45.96,17.0,30.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,69.0,76.0,47.59,66.54,68.57,49.25,582.0,601.0,49.2,253.0,265.0,48.84,42.0,42.0,50.00,329.0,336.0,49.47,13.0,18.0


In [288]:
df_players.iloc[:,120:140].head()

,MDGF%_y,LDCF_y,LDCA_y,LDCF%_y,LDGF_y,LDGA_y,LDGF%_y,On-Ice SH%_y,On-Ice SV%_y,PDO_y,Off. Zone Starts_y,Neu. Zone Starts_y,Def. Zone Starts_y,On The Fly Starts_y,Off. Zone Start %_y,Off. Zone Faceoffs_y,Neu. Zone Faceoffs_y,Def. Zone Faceoffs_y,Off. Zone Faceoff %_y,On Ice Situation_y
0,36.17,615.0,724.0,45.93,10.0,15.0,40.00,9.41,88.22,0.976,262.0,416.0,319.0,930.0,45.09,484.0,607.0,551.0,46.76,All Strengths
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All Strengths
2,36.17,615.0,724.0,45.93,10.0,15.0,40.00,9.41,88.22,0.976,262.0,416.0,319.0,930.0,45.09,484.0,607.0,551.0,46.76,All Strengths
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,All Strengths
4,41.94,606.0,472.0,56.22,13.0,15.0,46.43,9.89,87.76,0.976,241.0,345.0,269.0,709.0,47.25,438.0,533.0,519.0,45.77,All Strengths


In [289]:
df_players.iloc[:,140:160].head()

,Stats_y,Goals_y,Total Assists_y,First Assists_y,Second Assists_y,Total Points_y,IPP_y,Shots_y,SH%_y,ixG_y,iCF_y,iFF_y,iSCF_y,iHDCF_y,Rush Attempts_y,Rebounds Created_y,PIM_y,Total Penalties_y,Minor_y,Major_y
0,On-Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Individual,9.0,22.0,12.0,10.0,31.0,41.89,144.0,6.25,8.84,276.0,199.0,69.0,18.0,6.0,17.0,98.0,37.0,34.0,0.0
2,On-Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Individual,9.0,22.0,12.0,10.0,31.0,41.89,144.0,6.25,8.84,276.0,199.0,69.0,18.0,6.0,17.0,98.0,37.0,34.0,0.0
4,On-Ice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
df_players.iloc[:,160:].head()

,Misconduct_y,Penalties Drawn_y,Giveaways_y,Takeaways_y,Hits_y,Hits Taken_y,Shots Blocked_y,Faceoffs Won_y,Faceoffs Lost_y,Faceoffs %_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.0,7.0,58.0,21.0,93.0,25.0,143.0,4.0,4.0,50.00
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,7.0,58.0,21.0,93.0,25.0,143.0,4.0,4.0,50.00
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [291]:
# Group by 'Player' and 'Year'
grouped = df_players.groupby(['Player', 'Years'])

# Define a function to fill missing values in each group
def fill_missing_values(group):
    return group.fillna(method='ffill').fillna(method='bfill')

# Apply the function to each group
df_players = grouped.apply(fill_missing_values)

# Reset index if needed
df_players = df_players.reset_index(drop=True)

In [292]:
df_players.head()

,Unnamed: 0_x,Player,Team_x,Position_x,GP_x,TOI_x,CF_x,CA_x,CF%_x,FF_x,...,Misconduct_y,Penalties Drawn_y,Giveaways_y,Takeaways_y,Hits_y,Hits Taken_y,Shots Blocked_y,Faceoffs Won_y,Faceoffs Lost_y,Faceoffs %_y
0,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,...,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
1,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,...,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
2,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,...,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
3,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,...,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
4,776,A.J. Greer,COL,L,17,125.683333,88.0,129.0,40.55,68.0,...,1.0,7.0,2.0,3.0,30.0,16.0,5.0,1.0,2.0,33.33


In [293]:
for i in range (0, 170, 20):
    display(df_players.iloc[:,i:i+20].head())

,Unnamed: 0_x,Player,Team_x,Position_x,GP_x,TOI_x,CF_x,CA_x,CF%_x,FF_x,FA_x,FF%_x,SF_x,SA_x,SF%_x,GF_x,GA_x,GF%_x,xGF_x,xGA_x
0,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94,1.0,3.0,25.00,3.69,1.63
1,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94,1.0,3.0,25.00,3.69,1.63
2,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94,1.0,3.0,25.00,3.69,1.63
3,826,A.J. Greer,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94,1.0,3.0,25.00,3.69,1.63
4,776,A.J. Greer,COL,L,17,125.683333,88.0,129.0,40.55,68.0,100.0,40.48,54.0,73.0,42.52,4.0,2.0,66.67,3.45,5.19


,xGF%_x,SCF_x,SCA_x,SCF%_x,HDCF_x,HDCA_x,HDCF%_x,HDGF_x,HDGA_x,HDGF%_x,MDCF_x,MDCA_x,MDCF%_x,MDGF_x,MDGA_x,MDGF%_x,LDCF_x,LDCA_x,LDCF%_x,LDGF_x
0,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0,0.00,29.0,31.0,48.33,0.0
1,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0,0.00,29.0,31.0,48.33,0.0
2,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0,0.00,29.0,31.0,48.33,0.0
3,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0,0.00,29.0,31.0,48.33,0.0
4,39.92,46.0,54.0,46.0,17.0,26.0,39.53,2.0,1.0,66.67,29.0,28.0,50.88,1.0,0.0,100.00,35.0,60.0,36.84,1.0


,LDGA_x,LDGF%_x,On-Ice SH%_x,On-Ice SV%_x,PDO_x,Off. Zone Starts_x,Neu. Zone Starts_x,Def. Zone Starts_x,On The Fly Starts_x,Off. Zone Start %_x,Off. Zone Faceoffs_x,Neu. Zone Faceoffs_x,Def. Zone Faceoffs_x,Off. Zone Faceoff %_x,Years,On Ice Situation_x,Stats_x,Goals_x,Total Assists_x,First Assists_x
0,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,2016-2017,All Strengths,On-Ice,0.0,1.0,0.0
1,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,2016-2017,All Strengths,On-Ice,0.0,1.0,0.0
2,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,2016-2017,All Strengths,Individual,0.0,1.0,0.0
3,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,2016-2017,All Strengths,Individual,0.0,1.0,0.0
4,1.0,50.00,7.41,97.26,1.047,24.0,21.0,21.0,122.0,53.33,36.0,34.0,33.0,52.17,2017-2018,All Strengths,On-Ice,0.0,3.0,2.0


,Second Assists_x,Total Points_x,IPP_x,Shots_x,SH%_x,ixG_x,iCF_x,iFF_x,iSCF_x,iHDCF_x,Rush Attempts_x,Rebounds Created_x,PIM_x,Total Penalties_x,Minor_x,Major_x,Misconduct_x,Penalties Drawn_x,Giveaways_x,Takeaways_x
0,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0,0.0,2.0,2.0,1.0
1,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0,0.0,2.0,2.0,1.0
2,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0,0.0,2.0,2.0,1.0
3,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0,0.0,2.0,2.0,1.0
4,1.0,3.0,75.00,13.0,0.00,0.58,20.0,15.0,8.0,3.0,1.0,0.0,29.0,9.0,7.0,1.0,1.0,7.0,2.0,3.0


,Hits_x,Hits Taken_x,Shots Blocked_x,Faceoffs Won_x,Faceoffs Lost_x,Faceoffs %_x,Unnamed: 0_y,Team_y,Position_y,GP_y,TOI_y,CF_y,CA_y,CF%_y,FF_y,FA_y,FF%_y,SF_y,SA_y,SF%_y
0,4.0,6.0,3.0,0.0,0.0,-,826,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94
1,4.0,6.0,3.0,0.0,0.0,-,826,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94
2,4.0,6.0,3.0,0.0,0.0,-,826,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94
3,4.0,6.0,3.0,0.0,0.0,-,826,COL,L,5,68.283333,66.0,57.0,53.66,51.0,38.0,57.3,36.0,32.0,52.94
4,30.0,16.0,5.0,1.0,2.0,33.33,776,COL,L,17,125.683333,88.0,129.0,40.55,68.0,100.0,40.48,54.0,73.0,42.52


,GF_y,GA_y,GF%_y,xGF_y,xGA_y,xGF%_y,SCF_y,SCA_y,SCF%_y,HDCF_y,HDCA_y,HDCF%_y,HDGF_y,HDGA_y,HDGF%_y,MDCF_y,MDCA_y,MDCF%_y,MDGF_y,MDGA_y
0,1.0,3.0,25.00,3.69,1.63,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0
1,1.0,3.0,25.00,3.69,1.63,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0
2,1.0,3.0,25.00,3.69,1.63,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0
3,1.0,3.0,25.00,3.69,1.63,69.41,31.0,14.0,68.89,15.0,4.0,78.95,1.0,0.0,100.00,16.0,10.0,61.54,0.0,2.0
4,4.0,2.0,66.67,3.45,5.19,39.92,46.0,54.0,46.0,17.0,26.0,39.53,2.0,1.0,66.67,29.0,28.0,50.88,1.0,0.0


,MDGF%_y,LDCF_y,LDCA_y,LDCF%_y,LDGF_y,LDGA_y,LDGF%_y,On-Ice SH%_y,On-Ice SV%_y,PDO_y,Off. Zone Starts_y,Neu. Zone Starts_y,Def. Zone Starts_y,On The Fly Starts_y,Off. Zone Start %_y,Off. Zone Faceoffs_y,Neu. Zone Faceoffs_y,Def. Zone Faceoffs_y,Off. Zone Faceoff %_y,On Ice Situation_y
0,0.00,29.0,31.0,48.33,0.0,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,All Strengths
1,0.00,29.0,31.0,48.33,0.0,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,All Strengths
2,0.00,29.0,31.0,48.33,0.0,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,All Strengths
3,0.00,29.0,31.0,48.33,0.0,1.0,0.00,2.78,90.63,0.934,13.0,23.0,14.0,52.0,48.15,32.0,31.0,29.0,52.46,All Strengths
4,100.00,35.0,60.0,36.84,1.0,1.0,50.00,7.41,97.26,1.047,24.0,21.0,21.0,122.0,53.33,36.0,34.0,33.0,52.17,All Strengths


,Stats_y,Goals_y,Total Assists_y,First Assists_y,Second Assists_y,Total Points_y,IPP_y,Shots_y,SH%_y,ixG_y,iCF_y,iFF_y,iSCF_y,iHDCF_y,Rush Attempts_y,Rebounds Created_y,PIM_y,Total Penalties_y,Minor_y,Major_y
0,On-Ice,0.0,1.0,0.0,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0
1,Individual,0.0,1.0,0.0,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0
2,On-Ice,0.0,1.0,0.0,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0
3,Individual,0.0,1.0,0.0,1.0,1.0,100.00,8.0,0.00,1.28,11.0,10.0,7.0,4.0,0.0,2.0,4.0,2.0,2.0,0.0
4,On-Ice,0.0,3.0,2.0,1.0,3.0,75.00,13.0,0.00,0.58,20.0,15.0,8.0,3.0,1.0,0.0,29.0,9.0,7.0,1.0


,Misconduct_y,Penalties Drawn_y,Giveaways_y,Takeaways_y,Hits_y,Hits Taken_y,Shots Blocked_y,Faceoffs Won_y,Faceoffs Lost_y,Faceoffs %_y
0,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
1,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
2,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
3,0.0,2.0,2.0,1.0,4.0,6.0,3.0,0.0,0.0,-
4,1.0,7.0,2.0,3.0,30.0,16.0,5.0,1.0,2.0,33.33


In [294]:
cols_to_drop = df_players.filter(regex='_y$').columns
df_players = df_players.drop(columns=cols_to_drop)

In [295]:
df_players = df_players.drop(columns=['On Ice Situation_x', 'Stats_x', 'Unnamed: 0_x'])

In [296]:
df_players.columns = df_players.columns.str.replace('_x$', '', regex=True)

In [297]:
df_players.drop_duplicates(inplace=True)

In [298]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15281 entries, 0 to 61120
Data columns (total 83 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Player               15281 non-null  object 
 1   Team                 15281 non-null  object 
 2   Position             15281 non-null  object 
 3   GP                   15281 non-null  int64  
 4   TOI                  15281 non-null  float64
 5   CF                   15281 non-null  float64
 6   CA                   15281 non-null  float64
 7   CF%                  15281 non-null  object 
 8   FF                   15281 non-null  float64
 9   FA                   15281 non-null  float64
 10  FF%                  15281 non-null  object 
 11  SF                   15281 non-null  float64
 12  SA                   15281 non-null  float64
 13  SF%                  15281 non-null  object 
 14  GF                   15281 non-null  float64
 15  GA                   15281 non-null  floa

In [299]:
percent_columns = [col for col in df_players.columns if '%' in col]

# Clean the specified columns
for col in percent_columns:
    df_players[col] = df_players[col].str.replace(r'[^\d.]', '', regex=True)
    df_players[col] = pd.to_numeric(df_players[col], errors='coerce')

# Replace missing values with 0
df_players[percent_columns] = df_players[percent_columns].fillna(0)

# Check for any remaining missing values
missing_values = df_players.isnull().sum()
if missing_values.any():
    print(missing_values[missing_values > 0])
else:
    print("No missing values found.")

No missing values found.


In [300]:
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['FF%_Real'] = (df_players['FF'] / (df_players['FF'] + df_players['FA'])) * 100
#df_players['SF%_Real'] = (df_players['SF'] / (df_players['SF'] + df_players['SA'])) * 100
#df_players['GF%_Real'] = (df_players['GF'] / (df_players['GF'] + df_players['GA'])) * 100
#df_players['xGF%_Real'] = (df_players['xGF'] / (df_players['xGF'] + df_players['xGA'])) * 100
#df_players['SCF%_Real'] = (df_players['SCF'] / (df_players['SCF'] + df_players['SCA'])) * 100
#df_players['HDCF%_Real'] = (df_players['HDCF'] / (df_players['HDCF'] + df_players['HDCA'])) * 100
#df_players['HDGF%_Real'] = (df_players['HDGF'] / (df_players['HDGF'] + df_players['HDGA'])) * 100
#df_players['MDCF%_Real'] = (df_players['MDCF'] / (df_players['MDCF'] + df_players['MDCA'])) * 100
#df_players['LDCF%_Real'] = (df_players['LDCF'] / (df_players['LDCF'] + df_players['LDCA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100
#df_players['CF%_Real'] = (df_players['CF'] / (df_players['CF'] + df_players['CA'])) * 100

In [301]:
df_goalies = df_goalies[df_goalies['On Ice Situation'] == 'All Strengths']

In [302]:
df_goalies.head()

,Unnamed: 0,Player,Team,GP,TOI,Shots Against,Saves,Goals Against,SV%,GAA,...,LD Goals Against,LDSV%,LDGAA,LDGSAA,Rush Attempts Against,Rebound Attempts Against,Avg. Shot Distance,Avg. Goal Distance,Years,On Ice Situation
0,1,Dominik Hasek,DET,41,2350.066667,855,771,84,0.902,2.14,...,14,0.96,0.36,-0.37,45,61,36.29,28.90,2007-2008,All Strengths
1,2,Curtis Joseph,CGY,9,399.666667,181,164,17,0.906,2.55,...,3,0.96,0.45,-0.11,9,21,36.82,27.06,2007-2008,All Strengths
2,3,Olie Kolzig,WSH,54,3153.633333,1423,1270,153,0.892,2.91,...,23,0.964,0.44,1.38,52,126,37.56,21.47,2007-2008,All Strengths
3,4,Martin Brodeur,N.J,77,4613.166667,2088,1920,168,0.92,2.19,...,33,0.966,0.43,3.90,73,155,36.37,23.84,2007-2008,All Strengths
4,5,Chris Osgood,DET,43,2413.300000,976,892,84,0.914,2.09,...,21,0.948,0.52,-5.52,35,70,35.3,26.50,2007-2008,All Strengths


In [303]:
df_goalies = df_goalies.drop(columns=['On Ice Situation', 'Unnamed: 0'])

In [304]:
df_goalies.head()

,Player,Team,GP,TOI,Shots Against,Saves,Goals Against,SV%,GAA,GSAA,...,LD Saves,LD Goals Against,LDSV%,LDGAA,LDGSAA,Rush Attempts Against,Rebound Attempts Against,Avg. Shot Distance,Avg. Goal Distance,Years
0,Dominik Hasek,DET,41,2350.066667,855,771,84,0.902,2.14,-6.31,...,340,14,0.96,0.36,-0.37,45,61,36.29,28.90,2007-2008
1,Curtis Joseph,CGY,9,399.666667,181,164,17,0.906,2.55,-0.55,...,72,3,0.96,0.45,-0.11,9,21,36.82,27.06,2007-2008
2,Olie Kolzig,WSH,54,3153.633333,1423,1270,153,0.892,2.91,-23.70,...,610,23,0.964,0.44,1.38,52,126,37.56,21.47,2007-2008
3,Martin Brodeur,N.J,77,4613.166667,2088,1920,168,0.92,2.19,21.72,...,925,33,0.966,0.43,3.90,73,155,36.37,23.84,2007-2008
4,Chris Osgood,DET,43,2413.300000,976,892,84,0.914,2.09,4.68,...,381,21,0.948,0.52,-5.52,35,70,35.3,26.50,2007-2008


In [305]:
df_goalies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1592 entries, 0 to 6027
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Player                    1592 non-null   object 
 1   Team                      1592 non-null   object 
 2   GP                        1592 non-null   int64  
 3   TOI                       1592 non-null   float64
 4   Shots Against             1592 non-null   int64  
 5   Saves                     1592 non-null   int64  
 6   Goals Against             1592 non-null   int64  
 7   SV%                       1592 non-null   object 
 8   GAA                       1592 non-null   float64
 9   GSAA                      1592 non-null   float64
 10  xG Against                1592 non-null   float64
 11  HD Shots Against          1592 non-null   int64  
 12  HD Saves                  1592 non-null   int64  
 13  HD Goals Against          1592 non-null   int64  
 14  HDSV%        

In [306]:
object_columns = df_goalies.select_dtypes(include='object').columns
print(object_columns)

Index(['Player', 'Team', 'SV%', 'HDSV%', 'MDSV%', 'LDSV%',
       'Avg. Shot Distance', 'Avg. Goal Distance', 'Years'],
      dtype='object')


In [307]:
# Identify columns containing '%' and clean them
percent_columns = [col for col in df_goalies.columns if '%' in col]

for col in percent_columns:
    df_goalies[col] = df_goalies[col].str.replace(r'[^\d.]', '', regex=True)
    df_goalies[col] = pd.to_numeric(df_goalies[col], errors='coerce')

# Identify columns containing 'Avg' and clean them
avg_columns = [col for col in df_goalies.columns if 'Avg' in col]

for col in avg_columns:
    df_goalies[col] = df_goalies[col].str.replace(r'[^\d.]', '', regex=True)
    df_goalies[col] = pd.to_numeric(df_goalies[col], errors='coerce')

# Replace missing values with 0 for both sets of columns
df_goalies[percent_columns + avg_columns] = df_goalies[percent_columns + avg_columns].fillna(0)

# Check for any remaining missing values
missing_values = df_goalies.isnull().sum()
if missing_values.any():
    print(missing_values[missing_values > 0])
else:
    print("No missing values found.")

No missing values found.


In [310]:
df_goalies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1592 entries, 0 to 6027
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Player                    1592 non-null   object 
 1   Team                      1592 non-null   object 
 2   GP                        1592 non-null   int64  
 3   TOI                       1592 non-null   float64
 4   Shots Against             1592 non-null   int64  
 5   Saves                     1592 non-null   int64  
 6   Goals Against             1592 non-null   int64  
 7   SV%                       1592 non-null   float64
 8   GAA                       1592 non-null   float64
 9   GSAA                      1592 non-null   float64
 10  xG Against                1592 non-null   float64
 11  HD Shots Against          1592 non-null   int64  
 12  HD Saves                  1592 non-null   int64  
 13  HD Goals Against          1592 non-null   int64  
 14  HDSV%        

In [308]:
def save_to_csv(df, filename):
    if not os.path.isfile(filename):
        df.to_csv(filename, index=False)
    else:  
        df.to_csv(filename, mode='a', header=False, index=False)

In [309]:
player_advanced_stats_data_formatted = r"C:\Users\gabri\Documents\formatted_player_advanced_stats_data.csv"
save_to_csv(df_players, player_advanced_stats_data_formatted)

goalie_advanced_stats_data_formatted = r"C:\Users\gabri\Documents\formatted_goalie_advanced_stats_data.csv"
save_to_csv(df_goalies, goalie_advanced_stats_data_formatted)